In [ ]:
%matplotlib inline

import io, os, sys, types, datetime, pickle, warnings, time, operator
from colorama import Fore

warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import numpy as np
from numpy.linalg import eig, inv
import numpy.ma as ma

import math

from scipy import interpolate, spatial, stats, optimize


import scipy.stats as ss
import statsmodels.api as sa
import scikit_posthocs as sp

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float, filters, morphology, transform

from sklearn import linear_model
from sklearn import metrics

import itertools
# https://github.com/webermarcolivier/statannot

In [ ]:
module_path = os.getcwd()
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
module_path

In [ ]:
### import custom functions
import settings as settings
import summary_helper as my_help

In [ ]:
""" ============== Golbal variables and parameters =============="""
paths = settings.paths
matching_info = settings.matching_info
color_code =  matching_info.color_code
color_code[0] = 'gray'

palette = {
    'R3':color_code[3],
    'R4':color_code[4],
    'R3/R3':color_code[3],
    'R4/R4':color_code[4]
}

is_save = True
fig_format = 'png'
fig_res = 300

In [ ]:
""" ===== load data ===== """

In [ ]:
# ====== output data ====== #
folders, files = my_help.parse_folder_info(os.path.join(paths.data_prefix, 'output_datas'))
output_data_ctrl = {}
output_data_fz = {}
output_data_nic = {}
for file in files:
    pickle_in = open(os.path.join(paths.data_prefix, 'output_datas', file), "rb")
    %time data = pickle.load(pickle_in)
    if('ctrl' in file):
        for key in data.keys():
            output_data_ctrl[key] = data[key]
    elif('fz' in file):
        for key in data.keys():
            output_data_fz[key] = data[key]
    elif('nic' in file):
        for key in data.keys():
            output_data_nic[key] = data[key]
    del data

output_data_ctrl = my_help.sort_dic(output_data_ctrl, "keys", False)
output_data_fz = my_help.sort_dic(output_data_fz, "keys", False)
output_data_nic = my_help.sort_dic(output_data_nic, "keys", False)

In [ ]:

## controls
print("=== Control ===")
path = os.path.join(paths.data_prefix, 'output_data_ctrl.pickle')
pickle_in = open(path,"rb")
%time output_data_ctrl = pickle.load(pickle_in)

## Fzs
print("=== Fz ===")
path = os.path.join(paths.data_prefix, 'output_data_fz.pickle')
pickle_in = open(path,"rb")
%time output_data_fz = pickle.load(pickle_in)

## Nics
print("=== Nic ===")
path = os.path.join(paths.data_prefix, 'output_data_nic.pickle')
pickle_in = open(path,"rb")
%time output_data_nic = pickle.load(pickle_in)

In [ ]:
# ====== annotation files ====== #
### controls
filepath = os.path.join(paths.data_prefix, 'annots_df_ctrl.csv')
annots_df_ctrl = pd.read_csv(filepath)
annots_df_ctrl.fillna(0, inplace = True)

### Fzs
filepath = os.path.join(paths.data_prefix, 'annots_df_fz.csv')
annots_df_fz = pd.read_csv(filepath)
annots_df_fz.fillna(0, inplace = True)

### Nics
filepath = os.path.join(paths.data_prefix, 'annots_df_nic.csv')
annots_df_nic = pd.read_csv(filepath)
annots_df_nic.fillna(0, inplace = True)


In [ ]:
"""load and process summary files"""

In [ ]:
# ====== summary files ====== #
### controls
print("=== Control ===")
sum_df_ctrl = pd.read_csv(os.path.join(paths.data_prefix, 'summary_ctrl.csv'))
sum_df_ctrl = sum_df_ctrl.loc[~sum_df_ctrl['CategoryID'].isna(),:]
sum_df_ctrl[['TimeID', 'SampleID', 'bundle_no']] = sum_df_ctrl[['TimeID', 'SampleID', 'bundle_no']].astype('int64')
sum_df_ctrl_final = my_help.process_sum_df_mr(sum_df_ctrl, annots_df_ctrl, output_data_ctrl)
sum_df_ctrl_final.sort_values(by =['TimeID', 'SampleID', 'bundle_no'], inplace = True)

### Fzs
print("\n=== Fz ===")
sum_df_fz = pd.read_csv(os.path.join(paths.data_prefix, 'summary_fz.csv'))
sum_df_fz = sum_df_fz.loc[~sum_df_fz['CategoryID'].isna(),:]
sum_df_fz[['TimeID', 'SampleID', 'bundle_no']] = sum_df_fz[['TimeID', 'SampleID', 'bundle_no']].astype('int64')
sum_df_fz_final = my_help.process_sum_df(sum_df_fz, annots_df_fz, output_data_fz)
sum_df_fz_final.sort_values(by =['TimeID', 'SampleID', 'bundle_no'], inplace = True)

### Nics
print("\n=== Nic ===")
sum_df_nic = pd.read_csv(os.path.join(paths.data_prefix, 'summary_nic.csv'))
sum_df_nic = sum_df_nic.loc[~sum_df_nic['CategoryID'].isna(),:]
sum_df_nic[['TimeID', 'SampleID', 'bundle_no']] = sum_df_nic[['TimeID', 'SampleID', 'bundle_no']].astype('int64')
sum_df_nic_final = my_help.process_sum_df(sum_df_nic, annots_df_nic, output_data_nic)
sum_df_nic_final.sort_values(by =['TimeID', 'SampleID', 'bundle_no'], inplace = True)

In [ ]:
""" ===== plotting ===== """

In [ ]:
"""Figure 2"""

In [ ]:
my_help.generate_density_plot_figure(15, output_data_ctrl[(26, 21)], 
                                     is_save = is_save, channels = ['GFP'], gfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'Figure2', fig_format = fig_format, fig_res = fig_res)

In [ ]:
"""Figure 3A"""

In [ ]:
## 22 hrs
my_help.generate_density_plot_figure(5, output_data_ctrl[(22, 3)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'Figure3A_22hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 28 hrs
my_help.generate_density_plot_figure(6, output_data_ctrl[(28, 21)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.15, [0, 0.1]], rfp_cmap = [0.2, [0, 0.1, 0.2]],
                                     fig_name = 'Figure3A_28hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 32 hrs
my_help.generate_density_plot_figure(22, output_data_ctrl[(32, 13)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'Figure3A_32hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
"""Figure S2"""

In [ ]:
## 24 hrs
my_help.generate_density_plot_figure(4, output_data_ctrl[(24, 21)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'FigureS2_24hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 26 hrs
my_help.generate_density_plot_figure(1, output_data_ctrl[(26, 24)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2]], rfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'FigureS2_26hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 30 hrs
my_help.generate_density_plot_figure(1, output_data_ctrl[(30, 21)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'FigureS2_30hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## 36 hrs
my_help.generate_density_plot_figure(9, output_data_ctrl[(36, 21)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'FigureS2_36hrs', fig_format = fig_format, fig_res = fig_res)

In [ ]:
"""Figure 3B"""

In [ ]:
# ====== Figure ====== #
### set parameters
df_current = sum_df_ctrl_final
x_cat = 'TimeID'
y_cat = 'length_gc'
hue = 'type_plot'
df_plot = df_current[[x_cat, y_cat, hue]].dropna(how='any')
ylim = [0, 3]
ylabel = 'Relative Length (a.u.)'
xlabel = 'Time (hrs APF)'

### calculate T3/T4 range
times = len(df_current.groupby(x_cat).groups)
x = np.arange(times+2)-1
y = np.zeros((6, times+2))

y[0,1:-1] = df_current.groupby(x_cat).mean()['T4l_fromheel']
y[1,1:-1] = df_current.groupby(x_cat).mean()['T4h_fromheel']
y[2,1:-1] = df_current.groupby(x_cat).mean()['T4c_fromheel']
y[3,1:-1] = df_current.groupby(x_cat).mean()['T3l_fromheel']
y[4,1:-1] = df_current.groupby(x_cat).mean()['T3h_fromheel']
y[5,1:-1] = df_current.groupby(x_cat).mean()['T3c_fromheel']
y[:,0] = y[:,1]
y[:,-1] = y[:,-2]

### figure set-up
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 28
plt.rc('font', size=SMALL_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)   # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title

sns.set_style("white")

### plotting
fig, ax = plt.subplots(1,1, figsize = (12,8))
if('length' in y_cat):
    ax.plot(x, y[2,:], '--', color = color_code[4])
    ax.plot(x, y[5,:], '--', color = color_code[3])
    ax.fill_between(x, y[1,:], y[0,:], color=color_code[4], alpha=0.2)
    ax.fill_between(x, y[3,:], y[4,:], color=color_code[3], alpha=0.2)

sns.boxplot(x=x_cat, y=y_cat, data=df_plot, hue = hue, dodge = True, palette = palette, ax = ax)
sns.swarmplot(x=x_cat, y=y_cat, data=df_plot, color = '.25', hue = hue, dodge = True, ax = ax)
ax.set_ylim(ylim)
ax.set_ylabel(ylabel)
ax.set_xlabel(xlabel)
ax.tick_params(axis = 'y', labelleft = [0,1,2,3,4])

ax.legend(title = 'R type', loc='center left', bbox_to_anchor=(1.01, 0.9))

if(is_save):
    fig_name = f'Figure3B.{fig_format}'
    fig_save_path = os.path.join(paths.output_prefix, fig_name)
    plt.savefig(fig_save_path, dpi=fig_res, bbox_inches='tight', format = fig_format)

plt.show()

In [ ]:
# ====== linear fit of mean ====== #
X = np.array([24, 26, 28])
print("R3 extension speed:")
r_type = "R3"
y = np.zeros((3))
for i in range(3):
    y[i] = df_plot.groupby(hue).get_group(r_type).groupby(x_cat).mean()[y_cat][X[i]]
_, _, _ = my_help.fit_linear(X, y)

print("R4 extension speed:")
r_type = "R4"
y = np.zeros((3))
for i in range(3):
    y[i] = df_plot.groupby(hue).get_group(r_type).groupby(x_cat).mean()[y_cat][X[i]]
_, _, _ = my_help.fit_linear(X, y)

In [ ]:
# ====== p values ====== #

In [ ]:
r_type = "R3"
df_r = df_plot.groupby(hue).get_group(r_type)

data = [df_r.loc[ids, y_cat].values for ids in df_r.groupby(x_cat).groups.values()]
H, p = ss.kruskal(*data)
sp.posthoc_mannwhitney(df_r, val_col=y_cat, group_col=x_cat, p_adjust = 'holm')

In [ ]:
r_type = "R4"
df_r = df_plot.groupby(hue).get_group(r_type)

data = [df_r.loc[ids, y_cat].values for ids in df_r.groupby(x_cat).groups.values()]
H, p = ss.kruskal(*data)
sp.posthoc_mannwhitney(df_r, val_col=y_cat, group_col=x_cat, p_adjust = 'holm')

In [ ]:
"""Figure 3C"""

In [ ]:
# ====== Figure ====== #
### set parameters
x_cat = 'TimeID'
y_cat = 'angle_gc'
hue = "type_plot"
xlim = 8

colors1 = plt.cm.Reds(np.linspace(0, 1, 15))
colors1 = colors1[5:,:]

colors2 = plt.cm.YlOrBr(np.linspace(0, 1, 15))
colors2 = colors2[3:,:]

phi_unit = my_help.get_angle_unit_theory('phi_unit')

### calculate values
r3_control = sum_df_ctrl_final.groupby(hue).get_group('R3')
r4_control = sum_df_ctrl_final.groupby(hue).get_group('R4')

theta1 = r3_control.groupby(x_cat).mean()[y_cat].to_numpy()
width1 = r3_control.groupby(x_cat).std()[y_cat].to_numpy()

theta2 = r4_control.groupby(x_cat).mean()[y_cat].to_numpy()
width2 = r4_control.groupby(x_cat).std()[y_cat].to_numpy()

radii1 = np.arange(len(theta1))+1
radii2 = np.arange(len(theta2))+1

theta1 = np.flip(theta1)
radii1 = np.flip(radii1)
width1 = np.flip(width1)
theta2 = np.flip(theta2)
radii2 = np.flip(radii2)
width2 = np.flip(width2)

dTiCs = {}
target_grid_polar = my_help.get_target_grid_polar_summary(return_type = 'theory', dTiCs = dTiCs)

### figure set-up
SMALL_SIZE = 20
MEDIUM_SIZE = 20
BIGGER_SIZE = 28
plt.rc('font', size=SMALL_SIZE)         # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)   # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title



### plotting
fig = plt.figure(figsize=(12,8))
ax = fig.add_axes([0.1, 0.1, 0.75, 0.79], polar=True)

# plot reference line
ax.plot([0,0], [0,xlim], '--', color = 'Gray', linewidth = 1.5)
ax.plot([0,target_grid_polar[2,0]], [0,xlim], '--', color = 'Gray', linewidth = 1.5)
ax.plot([0,target_grid_polar[5,0]], [0,xlim], '--', color = 'Gray', linewidth = 1.5)

for i in range(len(radii1)):
    r = np.arange(-1, 1, 0.1)
    theta = np.zeros((r.shape)) + radii1[i]
    ax.plot(r,theta,'--', color = 'LightGray', linewidth = 2)

ax.bar(theta1, radii1, width=width1, bottom=0.0, color = colors1, alpha = 0.9)
ax.bar(theta2, radii2, width=width2, bottom=0.0, color = colors2, alpha=0.9)

ax.set_thetamin(-30)
ax.set_thetamax(30)
ax.set_rlim(0, xlim)

plt.yticks(ticks = [1, 2, 3, 4, 5, 6, 7], labels = ["22", "24", "26", "28", "30", "32", "36"])
plt.xticks(ticks = [-phi_unit, 0, phi_unit], labels = [1, 0, -1])
ax.set_xlabel("Time (hrs APF)")
ax.yaxis.set_label_position("right")
plt.ylabel("Relative Angle (a.u.)", labelpad=35)

ax.grid(False)

if(is_save):
    fig_name = f'Figure3C.{fig_format}'
    fig_save_path = os.path.join(paths.output_prefix, fig_name)
    plt.savefig(fig_save_path, dpi=fig_res, bbox_inches='tight', format = fig_format, facecolor='w')
plt.show()

In [ ]:
# ====== p values ====== #
x_cat = 'TimeID'
y_cat = 'angle_gc_mrr'
hue = 'type_plot'
df_plot = sum_df_ctrl_final[[x_cat, y_cat, hue]].dropna(how='any')
df_group = df_plot.groupby(x_cat)
for gp in df_group.groups.keys():
    print(f'{gp} hrs')
    my_help.p_value_3c(df_group.get_group(gp), hue, y_cat, [['R3', 'R4']], False, 'Mann-Whitney')

In [ ]:
"""Figure 4"""

In [ ]:
pert_cat = 'Fz'
time_id = 22

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info,  
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Fz'
time_id = 28

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Nic'
time_id = 22

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                     is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
pert_cat = 'Nic'
time_id = 28

if(pert_cat == 'Nic'):
    pert_type = 'R4/R4'
    pert_rtype = 4
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
elif(pert_cat == 'Fz'):
    pert_type = 'R3/R3'
    pert_rtype = 3
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)

pert_info = time_id, pert_cat, pert_type, pert_rtype
my_help.generate_summary_polar_figure(plot_df, pert_info, 
                                      is_save = is_save, fig_format = fig_format, fig_res = fig_res)

In [ ]:
# ====== p values: Fz, 22 hrs ====== #
pert_cat = 'Fz'
time_id = 22
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

y_cat = 'length_gc'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

y_cat = 'angle_gc_mrr'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

In [ ]:
# ====== p values: Fz, 28 hrs ====== #
pert_cat = 'Fz'
time_id = 28
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

y_cat = 'length_gc'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

y_cat = 'angle_gc_mrr'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

In [ ]:
# ====== p values: Nic, 22 hrs ====== #
pert_cat = 'Nic'
time_id = 22
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

y_cat = 'length_gc'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

y_cat = 'angle_gc_mrr'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

In [ ]:
# ====== p values: Nic, 28 hrs ====== #
pert_cat = 'Nic'
time_id = 28
x_cat = "type_plot"

if(pert_cat == 'Nic'):
    plot_df = sum_df_nic_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R4R4') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
elif(pert_cat == 'Fz'):
    plot_df = sum_df_fz_final.groupby('TimeID').get_group(time_id)
    criteria = (plot_df['type_bundle'] == 'R3R3') | (plot_df['type_bundle'] == 'R3R4')
    df_current = plot_df[criteria]
    

y_cat = 'length_gc'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

y_cat = 'angle_gc_mrr'
my_help.p_value_4(df_current, x_cat, y_cat, pert_cat, time_id)

In [ ]:
"""Figure S3"""

In [ ]:
## Fz 22 hrs, ctrl.
my_help.generate_density_plot_figure(7, output_data_fz[(22, 1)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2]], rfp_cmap = [0.25, [0, 0.1, 0.2]],
                                     fig_name = 'FigureS3_Fz_22hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Fz 22 hrs, pert.
my_help.generate_density_plot_figure(1, output_data_fz[(22, 5)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'FigureS3_Fz_22hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Fz 28 hrs, ctrl.
my_help.generate_density_plot_figure(36, output_data_fz[(28, 26)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'FigureS3_Fz_28hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Fz 28 hrs, pert.
my_help.generate_density_plot_figure(1, output_data_fz[(28, 26)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.2, [0, 0.1, 0.2]], rfp_cmap = [0.15, [0, 0.1]],
                                     fig_name = 'FigureS3_Fz_28hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Nic 22 hrs, ctrl.
my_help.generate_density_plot_figure(2, output_data_nic[(22, 2)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.4, [0, 0.2, 0.4]],
                                     fig_name = 'FigureS3_Nic_22hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Nic 22 hrs, pert.
my_help.generate_density_plot_figure(1, output_data_nic[(22, 2)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.3, [0, 0.1, 0.2, 0.3]], rfp_cmap = [0.25, [0, 0.1, 0.2]],
                                     fig_name = 'FigureS3_Nic_22hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Nic 28 hrs, ctrl.
my_help.generate_density_plot_figure(5, output_data_nic[(28, 29)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.15, [0, 0.15]], rfp_cmap = [0.25, [0, 0.1, 0.2]],
                                     fig_name = 'FigureS3_Nic_28hrs_ctrl', fig_format = fig_format, fig_res = fig_res)

In [ ]:
## Nic 28 hrs, pert.
my_help.generate_density_plot_figure(1, output_data_nic[(28, 23)], 
                                     is_save = is_save, channels = ['GFP', 'RFP'], 
                                     gfp_cmap = [0.25, [0, 0.1, 0.2]], rfp_cmap = [0.35, [0, 0.1, 0.2, 0.3]],
                                     fig_name = 'FigureS3_Nic_28hrs_pert', fig_format = fig_format, fig_res = fig_res)

In [ ]:
"======= Figure S4 ======="

In [ ]:
my_help.mutual_repulsion_regression_plot(sum_df_ctrl_final, annots_df_ctrl, is_save = is_save, fig_name = 'FigureS4', fig_format = fig_format, fig_res = fig_res)